In [35]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# Step 1: Create sample data with an extra movie only rated by User3
data = [
    {'User': 'User1', 'Movie': 'Interception', 'Rating': 5},
    {'User': 'User1', 'Movie': 'Interstellar', 'Rating': 4},
    {'User': 'User1', 'Movie': 'Stranger Things', 'Rating': 4},
    {'User': 'User2', 'Movie': 'Interception', 'Rating': 4},
    {'User': 'User2', 'Movie': 'Stranger Things', 'Rating': 5},
    {'User': 'User3', 'Movie': 'Interception', 'Rating': 5},
    {'User': 'User3', 'Movie': 'Stranger Things', 'Rating': 3},
    {'User': 'User3', 'Movie': 'Dark', 'Rating': 5},  # Only User3 rated this
    {'User': 'User4', 'Movie': 'Interstellar', 'Rating': 3},
    {'User': 'User4', 'Movie': 'Stranger Things', 'Rating': 5}
]

# Step 2: Create DataFrame
df = pd.DataFrame(data)

# Step 3: Create user-movie matrix
user_movie_matrix = df.pivot_table(index='User', columns='Movie', values='Rating').fillna(0)

# Step 4: Build the Nearest Neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_movie_matrix)

# Step 5: Choose a sample user (e.g., User1) and get similar users
sample_user = 'User1'
sample_user_index = user_movie_matrix.index.get_loc(sample_user)
distances, indices = model.kneighbors(user_movie_matrix.iloc[[sample_user_index]], n_neighbors=3)

# Step 6: Get recommended movies from similar users
similar_users = user_movie_matrix.index[indices.flatten()[1:]]  # Exclude the first (it's the user itself)

# Get movies watched by similar users but not by the sample user
sample_user_movies = set(df[df['User'] == sample_user]['Movie'])
recommended_movies = {}

for sim_user in similar_users:
    sim_user_movies = df[df['User'] == sim_user]
    for _, row in sim_user_movies.iterrows():
        if row['Movie'] not in sample_user_movies:
            if row['Movie'] not in recommended_movies:
                recommended_movies[row['Movie']] = row['Rating']
            else:
                recommended_movies[row['Movie']] = max(recommended_movies[row['Movie']], row['Rating'])

# Step 7: Show recommendations
print(f"Similar users to {sample_user}: {list(similar_users)}\n")
print(f"Top movie recommendations for {sample_user}:")

if recommended_movies:
    for movie, rating in sorted(recommended_movies.items(), key=lambda x: -x[1]):
        print(f"{movie} (Predicted rating: {rating})")
else:
    print("No new recommendations available.")


Similar users to User1: ['User2', 'User4']

Top movie recommendations for User1:
No new recommendations available.


In [37]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors

# fixed data with spelling corrected
data = {
    'User': ['User1', 'User1', 'User1', 
             'User2', 'User2', 
             'User3', 'User3', 'User3', 
             'User4', 'User4',
             'User5', 'User5', 'User5', 'User5'],
    
    'Movie': ['Interception', 'Interstellar', 'Stranger Things',
              'Interception', 'Interstellar',
              'Interception', 'Stranger Things', 'Breaking Bad',
              'Interstellar', 'Stranger Things',
              'Dark', 'Interstellar', 'Money Heist', 'Interception'],
    
    'Rating': [5, 4, 5, 
               4, 5, 
               5, 4, 5,
               4, 5,
               5, 3, 4, 2]
}

df = pd.DataFrame(data)

# create user-item matrix
user_movie_matrix = df.pivot_table(index='User', columns='Movie', values='Rating').fillna(0)

# train Nearest Neighbors model
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_movie_matrix)

# get similar users
sample_user_index = list(user_movie_matrix.index).index('User1')
distances, indices = model.kneighbors([user_movie_matrix.iloc[sample_user_index]], n_neighbors=3)

# similar users
similar_users = user_movie_matrix.index[indices.flatten()[1:]]
print(f"Similar users to User1: {list(similar_users)}\n")

# get movies similar users rated that User1 hasn't
user1_movies = set(df[df['User'] == 'User1']['Movie'])
similar_users_movies = df[df['User'].isin(similar_users)]

# filter out movies User1 already watched
recommendations = similar_users_movies[~similar_users_movies['Movie'].isin(user1_movies)]
top_recommendations = recommendations.groupby('Movie')['Rating'].mean().sort_values(ascending=False).head(3)

# display
if not top_recommendations.empty:
    print("Top movie recommendations for User1:")
    for movie in top_recommendations.index:
        print(f"- {movie}")
else:
    print("Top movie recommendations for User1:\nNo new recommendations available.")


Similar users to User1: ['User4', 'User2']

Top movie recommendations for User1:
No new recommendations available.


D:\5th-Semester\jupyter\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
